In [1]:
# ========== ----- ========== Import Libraries ========== ----- ========== #

from LDP import LDP_TOP
from LDP import train
import sklearn
print(sklearn.__version__)

import numpy as np
import dlib
import cv2
import os
import re
import json
import numpy as np
import pandas as pd

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from joblib import dump, load

import pickle

# ========== ----- ========== End ========== ----- ========== #

1.2.2


In [18]:
# ========== ----- ========== Load SVM Model ========== ----- ========== #

# Load the saved model from the file
rbf_faceSwap = load('models/cf40/RBF-FaceSwap.joblib')
linear_faceSwap = load('models/cf40/Linear-FaceSwap.joblib')

# ========== ----- ========== End ========== ----- ========== #

In [4]:
# ========== ----- ========== Reading Test Data ========== ----- ========== #

# Reading Testing Data
test_frame_folders = [
    # 'faceforensics_dataset_test/original_sequences',
    # 'faceforensics_dataset_test/manipulated_sequences/Deepfakes',
    'faceforensics_dataset_test/manipulated_sequences/Face2Face',
    'faceforensics_dataset_test/manipulated_sequences/FaceSwap',
    'faceforensics_dataset_test/manipulated_sequences/NeuralTextures'
]

# ========== ----- ========== End ========== ----- ========== #

In [46]:
# ========== ----- ========== Feature Extraction ========== ----- ========== #

# Face detector
detector = dlib.get_frontal_face_detector()

list_of_test_LDP = []
list_of_test_labels = []

index = 0
for test_frame_folder in test_frame_folders:
    list_of_test_data = [f for f in os.listdir(
        test_frame_folder) if f.endswith('.mp4')]
    
    print(test_frame_folder, 'videos: ', len(list_of_test_data))
    for vid in list_of_test_data:
        list_of_test_LDP.append([])
        frames = []
        # Video capturing constructor.
        cap = cv2.VideoCapture(os.path.join(test_frame_folder, vid))

        # (CAP_PROP_FPS) Returns frame rate of the video (#frames / second).
        frameRate = cap.get(5)
        while cap.isOpened():  # Returns true if video capturing has been initialized already.
            # (CAP_PROP_POS_MSEC) Current position of the video file in milliseconds or video capture timestamp.
            frameId = cap.get(1)
            ret, frame = cap.read()  # The methods/functions combine VideoCapture::grab and VideoCapture::retrieve in one call. This is the most convenient method for reading video files or capturing data from decode and return the just grabbed frame. If no frames has been grabbed (camera has been disconnected, or there are no more frames in video file), the methods return false and the functions return NULL pointer.
            if ret != True:
                break

            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()

                crop_img = frame[y1:y2, x1:x2]
                if crop_img is not None and crop_img.size > 0:
                    crop_img = cv2.resize(crop_img, (128, 128))
                    gray_scale_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                    frames.append(gray_scale_img)

            # d = 3 seconds.
            if frameId % ((int(frameRate)+1)*3) == 0:
                if (len(frames) > 0):
                    frames_ldp = LDP_TOP(np.array(frames).astype(np.float64))
                    list_of_test_LDP[index].append(frames_ldp)

                frames = []
        index += 1

with open('features/Face2Face-FaceSwap-NeuralTextures-Test.pkl', 'wb') as f:
    pickle.dump(list_of_test_LDP, f)

# ========== ----- ========== End ========== ----- ========== #


faceforensics_dataset_test/manipulated_sequences/Face2Face videos:  140
faceforensics_dataset_test/manipulated_sequences/FaceSwap videos:  140
faceforensics_dataset_test/manipulated_sequences/NeuralTextures videos:  140


In [13]:
# ========== ----- ========== Feature Loading ========== ----- ========== #

# Loading the list_of_train_LDP
with open('features/cf40/list_of_test_LDP.pkl', 'rb') as f:
    test_features = pickle.load(f)

# ========== ----- ========== End ========== ----- ========== #

In [15]:
# ========== ----- ========== Prepare Features ========== ----- ========== #

# Deepfakes
list_of_test_LDP = test_features[:140] + \
    test_features[420:560]

# ========== ----- ========== End ========== ----- ========== #


In [19]:
# ========== ----- ========== SVM Predictions ========== ----- ========== #

rbf_faceSwap_predictions = []
linear_faceSwap_predictions = []

for vid in list_of_test_LDP:
    y_pred = rbf_faceSwap.predict(vid)
    # print(y_pred)
    if np.count_nonzero(y_pred == 1) >= np.count_nonzero(y_pred == 0):
        rbf_faceSwap_predictions.append(1)
    else:
        rbf_faceSwap_predictions.append(0)
    # ============================================= #
    
    y_pred = linear_faceSwap.predict(vid)
    # print(y_pred)
    if np.count_nonzero(y_pred == 1) >= np.count_nonzero(y_pred == 0):
        linear_faceSwap_predictions.append(1)
    else:
        linear_faceSwap_predictions.append(0)

# ========== ----- ========== End ========== ----- ========== #


In [20]:
# ========== ----- ========== SVM Accuracy ========== ----- ========== #

labels = [0]*140 + [1]*140

# calculate the accuracy score and print it out
accuracy = accuracy_score(labels, rbf_faceSwap_predictions)
print("RBF Accuracy:", accuracy)

# calculate the confusion matrix and print it out
cm = confusion_matrix(labels, rbf_faceSwap_predictions)
print("RBF Confusion matrix:\n", cm, "\n")

# ================================================ #

# calculate the accuracy score and print it out
accuracy = accuracy_score(labels, linear_faceSwap_predictions)
print("Linear Accuracy:", accuracy)

# calculate the confusion matrix and print it out
cm = confusion_matrix(labels, linear_faceSwap_predictions)
print("Linear Confusion matrix:\n", cm)

# ========== ----- ========== End ========== ----- ========== #


RBF Accuracy: 0.6535714285714286
RBF Confusion matrix:
 [[133   7]
 [ 90  50]] 

Linear Accuracy: 0.7321428571428571
Linear Confusion matrix:
 [[ 96  44]
 [ 31 109]]
